### Use the influence_data data set or portions of it to create a (multiple) directed network(s) of musical influence, where influencers are connected to followers. Develop parameters that capture ‘music influence’ in this network. Explore a subset of musical influence by creating a subnetwork of your directed influencer network. Describe this subnetwork. What do your ‘music influence’ measures reveal in this subnetwork?

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm import tqdm


In [3]:
# load data into DataFrames
artists = pd.read_csv('data_by_artist.csv')
years = pd.read_csv('data_by_year.csv')
influence = pd.read_csv('influence_data.csv')
full = pd.read_csv('full_music_data.csv')

# scale tempo columns in each df
scaleAttr(full, 'tempo')
scaleAttr(artists, 'tempo')
scaleAttr(years, 'tempo')

# no dups
artists = artists.drop_duplicates(subset='artist_name', keep='last')

NameError: name 'scaleAttr' is not defined

In [ ]:
def scaleAttr(df, attr):
    """scales the attribute attr of the dataframe df"""
    floor = min(df[attr])
    span = max(df[attr]) - floor
    df[attr] = [(x-floor)/span for x in df[attr]]

In [ ]:
#remove songs by artists that arent in the artists table
#duplicate songs w/ more than one artist
#takes my computer about a minute to run
#can ignore the warnings about setting a copy of a slice from a dataframe
allArtists = list(artists['artist_id'])
newdict = {}

full['artists_id'] = [int(x.split(',')[0][1:-1]) if len(x.split(',')) == 1 else [int(y) for y in x[1:-1].split(',')] for x in list(full['artists_id'])]
full['artist_names'] = [ast.literal_eval(x) if len(ast.literal_eval(x)) > 1 else ast.literal_eval(x)[0] for x in full['artist_names']]
scaleAttr(full, 'tempo')

last_id = 0
row = 0
for i in tqdm(range(len(full))): 
    thisRow = full.iloc[i,:]
    artIDs = thisRow['artists_id']
    artNames = thisRow['artist_names']
    
    if type(artIDs) == int:
        if artIDs == last_id: 
            thisRow['artists_id'] = artIDs
            thisRow['artist_names'] = artNames
            newdict[row] = thisRow.to_dict()
            row+=1
            last_id = artIDs
        else:
            if artIDs in allArtists:
                thisRow['artists_id'] = artIDs
                thisRow['artist_names'] = artNames
                newdict[row] = thisRow.to_dict()
                row+=1
                last_id = artIDs
                
    else:
        for j in range(len(artIDs)):
            artID = int(artIDs[j])
            artName = str(artNames[j])
            
            if artID in allArtists:
                thisRow['artists_id'] = artID
                thisRow['artist_names'] = artName
                newdict[row] = thisRow.to_dict()
                row+=1
    i+=1

full = pd.DataFrame.from_dict(newdict,'index')

artistGenres = {} #adds song genre to full
for i in tqdm(range(len(influence))):
    row = influence.iloc[i]
    influencer = row['influencer_id']
    follower = row['follower_id']
    infGenre = row['influencer_main_genre']
    folGenre = row['follower_main_genre']
    artistGenres[influencer] = infGenre
    artistGenres[follower] = folGenre

def getGenre(row):
    artist = row['artists_id']
    if type(artist) == list:
        print(row)
    try:
        genre = artistGenres[artist]
    except KeyError:
        genre = 'N/A'
    return genre

full['genre'] = full.apply(lambda row: getGenre(row), axis = 1)

full


In [4]:
influence

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990
...,...,...,...,...,...,...,...,...
42765,580300,Sufjan Stevens,Pop/Rock,1990,3661738,Rosemary & Garlic,Pop/Rock,2010
42766,261309,Vybz Kartel,Reggae,2000,3670556,Trinidad Cardona,R&B;,2010
42767,467203,Michael Jackson,R&B;,1960,3670556,Trinidad Cardona,R&B;,2010
42768,2518003,Popcaan,Reggae,2000,3670556,Trinidad Cardona,R&B;,2010


In [10]:
# list containing all artist names
names = []
for i in artists['artist_name']:
    names.append(i)

# reset index in the influence df
influence = influence[influence['influencer_name'].isin(names)]
influence = influence[influence['follower_name'].isin(names)]
influence = influence.reset_index(drop=True)


   influencer_id influencer_name influencer_main_genre  \
0         759491   The Exploited              Pop/Rock   
1          25462          Tricky            Electronic   
2          66915       Bob Dylan              Pop/Rock   
3          71209   Leonard Cohen              Pop/Rock   
4          91438    The Gun Club              Pop/Rock   

   influencer_active_start  follower_id   follower_name follower_main_genre  \
0                     1980           74  Special Duties            Pop/Rock   
1                     1990          335       PJ Harvey            Pop/Rock   
2                     1960          335       PJ Harvey            Pop/Rock   
3                     1950          335       PJ Harvey            Pop/Rock   
4                     1980          335       PJ Harvey            Pop/Rock   

   follower_active_start  
0                   1980  
1                   1990  
2                   1990  
3                   1990  
4                   1990  


In [81]:
# instantiate directed graph
influenceGraph = nx.DiGraph()

In [88]:
# add edges to directed graph
# measure influence 
songStats = ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'instrumentalness', 'speechiness']        

baseline = .9136464572663171 #from bullet 2 - average cosine similarity between a song and the era its from

idDict = {}
for i in tqdm(range(len(artists))):
    try:
        idDict[artists['artist_id'][i]] = artists['artist_name'][i]
    except KeyError:
        pass

influence = pd.read_csv('influence_data.csv')
iGraph = nx.DiGraph()

for artist in artists['artist_id']:
    iGraph.add_node(artist, weight = 0)
    
for i in tqdm(range(len(influence))):
    influencer = influence["influencer_id"][i]
    follower = influence['follower_id'][i]
    
    iGraph.add_edge(influencer,follower)    

arts = []
artnames = []
scores = []
otherScores = []
for node in tqdm(iGraph.nodes()):
    infStats = artists[artists['artist_id'] == node][songStats].to_numpy()
    children = list(iGraph.successors(node))
    total = 0
    unweighted = 0
    for child in children:
        folStats = artists[artists['artist_id'] == child][songStats].to_numpy()
        if len(folStats) == 0 or len(infStats) == 0:
            continue
        similarity = float(cosine_similarity(infStats, folStats))
        total += iGraph.out_degree[child]*(similarity/baseline)**2
        unweighted += iGraph.out_degree[child]
        
    arts.append(node)
    try:
        artnames.append(idDict[node])
    except KeyError:
        artnames.append('Unknown')
    scores.append(total)
    otherScores.append(unweighted)
    
   
totalInfluence = pd.DataFrame({'artist_name':artnames, 'artist_id':arts, 'influence_score':scores, 'unweighted_score':otherScores})
totalInfluence = totalInfluence.reset_index(drop=True)
totalInfluence = totalInfluence.sort_values(by=['influence_score'], ascending = False)
totalInfluence[:10]

100%|██████████████████████████████████████████████████████████████████████████████| 5853/5853 [01:01<00:00, 95.28it/s]


,artist_name,artist_id,influence_score,unweighted_score
15,The Beatles,754032,12043.015616,10948
4,Bob Dylan,66915,8561.607765,7823
5,The Rolling Stones,894465,7139.703708,6308
128,Chuck Berry,120521,6361.824726,5866
7,Elvis Presley,180228,4905.473305,4748
167,Little Richard,824022,4839.792842,4273
24,The Who,577627,4586.847910,4001
138,Jimi Hendrix,354105,4583.535579,4222
199,The Velvet Underground,840402,4486.019824,4276
35,The Kinks,100160,4475.013863,3990
